In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# from selenium.webdriver.chrome.options import Options

# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Runs Chrome in headless mode.
# driver = webdriver.Chrome(options=chrome_options, service=Service(ChromeDriverManager().install()))


In [13]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://robinhood.com/login")

# Wait for the page to load (you might need to add explicit waits)
driver.implicitly_wait(3)

wait = WebDriverWait(driver, 3)

In [14]:
# userlogin = wait.until(EC.visibility_of_element_located((By.XPATH, "//input[@name='username']")))
# userlogin.send_keys("perevicius.mykolas@gmail.com")


In [15]:
# userlogin = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "input[name='password']")))
# userlogin.send_keys("")

In [16]:

# Using the name attribute for the checkbox
keep_me_logged_in_checkbox = driver.find_element(By.CSS_SELECTOR, "input[name='long_session']")
driver.execute_script("arguments[0].click();", keep_me_logged_in_checkbox)



In [17]:
login_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
driver.execute_script("arguments[0].click();", login_button)


In [8]:
# Security Code

In [9]:
# /account/history
# important for ALL data, and perhaps real time trades

In [10]:
# /account/reports-statements/activity-reports
# important for latest

In [69]:
driver.get("https://robinhood.com/account/reports-statements/individual")
#download all monthly reports


In [70]:
from selenium.common.exceptions import TimeoutException

import os
import time 

download_directory = "C:/Users/miciu/Downloads"  

download_links = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[download]")))

num_downloads = 0

for link in download_links:
    try:
        link.click()  # Initiate the download
        print("Download initiated...")
        num_downloads += 1

        # You might need to add a short wait here to allow the download to start

    except TimeoutException:
        print(f"Download timed out for: {link.get_attribute('href')}")  # Handle potential timeouts 


Download initiated...
Download initiated...
Download initiated...
Download initiated...
Download initiated...
Download initiated...
Download initiated...
Download initiated...
Download initiated...
Download initiated...


In [80]:
def get_recent_files(download_dir, num_files):
    """Returns a list of 'num_files' most recently downloaded files (excluding hidden and '.parts')."""
    list_of_files = os.listdir(download_dir)
    filtered_files = [
        f for f in list_of_files
        if not f.endswith('.parts') and os.path.isfile(os.path.join(download_dir, f))
    ]
    filtered_files.sort(key=os.path.getctime, reverse=True)
    return [os.path.join(download_dir, filename) for filename in filtered_files[:num_files]]


recent_pdfs = get_recent_files(download_directory, num_downloads)
print(recent_pdfs)

FileNotFoundError: [WinError 2] The system cannot find the file specified: '.~lock.IGExport_mykolasp.jpeg_500_followers.csv#'

In [ ]:

# Process each downloaded PDF
for pdf_path in recent_pdfs:
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        # ... rest of your PDF processing code ...

In [20]:
import PyPDF2

# Open the PDF file
with open(r'C:\Users\miciu\Downloads\634f4556-3c92-40ba-8875-89ecd4697cdf.pdf', 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    
    # If the PDF is encrypted, you'll need the password to decrypt it
    # reader.decrypt('password')
    
    # Initialize an empty text string
    text = ""

    # Iterate over each page and extract text
    for page in reader.pages:
        text += page.extract_text()

print(text)


   Options  Equities  Cash and Cash Equivalents
* “Total Securities” includes those securities held in your account as well as those on loan through Stock Lending (identified herein as “Loaned Securities” or “Loaned Stocks”). The provisions of the Securities Investor Protection Act of 
1970 may not protect you with respect to Loaned Securities.
As of the closing date of this statement, the annual percentage yield (APY) for customers enrolled in the brokerage cash sweep program is 1.50% for customers who are not subscribed to Robinhood Gold, 5.00% for Robinhood Gold 
subscribers, or 5.25% for Robinhood Gold subscribers enrolled in the boosted rate promotion. Program banks pay interest on your swept cash, minus any fees paid to Robinhood. Neither Robinhood Financial LLC nor any of its affiliates are 
banks. Investment products are not FDIC-insured, not bank guaranteed, and may lose value. Uninvested cash swept to the program banks is eligible for FDIC insurance up to applicable limits, s

In [44]:
#todo: combine all into one regex
#todo: scan account activity not just Portfolio, income, expense, and account summary and portfolio allocation

import re

extracted_text = text
# Define a dictionary to store the parsed data
# Compile regular expressions for the different data fields
# Regular expressions for data patterns
data_patterns = {
    "Brokerage Cash Balance": r"Brokerage Cash Balance \$([-\d,.]+) \$([-\d,.]+)",
    "Deposit Sweep Balance": r"Deposit Sweep Balance \$([-\d,.]+) \$([-\d,.]+)",
    "Total Securities": r"Total Securities\* \$([-\d,.]+) \(\$([-\d,.]+)\)",
    "Portfolio Value": r"Portfolio Value \$([-\d,.]+) \(\$([-\d,.]+)\)",
    "Cash and Cash Equivalents": r"Cash and Cash Equivalents\s*([\d.]+)%",
    "Equities": r"Equities\s*([\d.]+)%",
    "Options": r"Options\s*([\d.]+)%",
    "Dividends": r"Dividends \$([-\d,.]+) \$([-\d,.]+)",
    "Capital Gains Distributions": r"Capital Gains Distributions \$([-\d,.]+) \$([-\d,.]+)",
    "Interest Earned": r"Interest Earned \$([-\d,.]+) \$([-\d,.]+)",
    "Stock Lending": r"Stock Lending \$([-\d,.]+) \$([-\d,.]+)"
}

extracted_data = {}

for key, pattern in data_patterns.items():
    matches = re.search(pattern, extracted_text, re.DOTALL)
    if matches:
        values = [matches.group(i) for i in range(1, matches.lastindex + 1)]
        extracted_data[key] = values
    else:
        extracted_data[key] = None

# Formatting and printing the output
print("Account Summary\nOpening Balance / Closing Balance")
print(f"Brokerage Cash Balance: ${extracted_data['Brokerage Cash Balance'][0]} / ${extracted_data['Brokerage Cash Balance'][1]}")
print(f"Deposit Sweep Balance: ${extracted_data['Deposit Sweep Balance'][0]} / ${extracted_data['Deposit Sweep Balance'][1]}")
print(f"Total Securities*: ${extracted_data['Total Securities'][0]} / (${extracted_data['Total Securities'][1]})")
print(f"Portfolio Value: ${extracted_data['Portfolio Value'][0]} / (${extracted_data['Portfolio Value'][1]})\n")

print("Portfolio Allocation")
print(f"Cash and Cash Equivalents: {extracted_data['Cash and Cash Equivalents'][0]}%")
print(f"Equities: {extracted_data['Equities'][0]}%")
print(f"Options: {extracted_data['Options'][0]}%\n")

print("Income and Expense Summary\nThis Period / Year to Date")
print(f"Dividends: ${extracted_data['Dividends'][0]} / ${extracted_data['Dividends'][1]}")
print(f"Capital Gains Distributions: ${extracted_data['Capital Gains Distributions'][0]} / ${extracted_data['Capital Gains Distributions'][1]}")
print(f"Interest Earned: ${extracted_data['Interest Earned'][0]} / ${extracted_data['Interest Earned'][1]}")
print(f"Stock Lending: ${extracted_data['Stock Lending'][0]} / ${extracted_data['Stock Lending'][1]}")
print("-----------------------------")

start_index = extracted_text.find("Portfolio Summary\nSecurities Held in Account Sym/Cusip Acct Type Qty Price Mkt Value Est. Dividend Yield % of Total Portfolio")
if start_index != -1:
    # Slice the text from the start of the portfolio summary onward
    portfolio_section = text[start_index:]

    # Improved regex pattern
    pattern = r"([A-Z]{3,5}) (\d{2}/\d{2}/\d{4}) (Put|Call) \$(\d+,\d+\.\d{2}|\d+\.\d{2})\nEstimated Yield: (\d+\.\d{2})%([A-Z]{3,5}) (\w+) (\d+\w*?) \$(\d+\.\d{2}) (\$?-?\d+,\d+\.\d{2}|\$?-?\d+\.\d{2}|\(\$\d+,\d+\.\d{2}\)) (\$?-?\d+,\d+\.\d{2}|\$?-?\d+\.\d{2}|\(\$\d+,\d+\.\d{2}\)) (\d+\.\d{2})%"

    matches = list(re.finditer(pattern, portfolio_section, re.MULTILINE))

    for match in matches:
        print(f"Security Name: {match.group(1)}")
        print(f"Date: {match.group(2)}")
        print(f"Type: {match.group(3)}")
        print(f"Price: {match.group(4)}")
        print(f"Estimated Yield: {match.group(5)}%")
        print(f"Security Name Repeat: {match.group(6)}")
        print(f"Account Type: {match.group(7)}")
        print(f"Quantity: {match.group(8)}")
        print(f"Price: {match.group(9)}")
        print(f"Market Value: {match.group(10)}")
        print(f"Market Value: {match.group(11)}")
        print(f"Portfolio Percentage: {match.group(12)}%")
        print("-----------------------------")


else:
    print("Portfolio Summary section not found.")


Account Summary
Opening Balance / Closing Balance
Brokerage Cash Balance: $0.00 / $26.94
Deposit Sweep Balance: $260.25 / $0.00
Total Securities*: $0.00 / ($435.00)
Portfolio Value: $260.25 / ($408.06)

Portfolio Allocation
Cash and Cash Equivalents: 0.16%
Equities: 0.00%
Options: 99.84%

Income and Expense Summary
This Period / Year to Date
Dividends: $0.00 / $0.00
Capital Gains Distributions: $0.00 / $0.00
Interest Earned: $0.00 / $0.01
Stock Lending: $0.00 / $0.00
-----------------------------
Security Name: GETY
Date: 03/15/2024
Type: Put
Price: 4.00
Estimated Yield: 0.00%
Security Name Repeat: GETY
Account Type: Margin
Quantity: 2
Price: 0.10
Market Value: $20.00
Market Value: $0.00
Portfolio Percentage: 0.12%
-----------------------------
Security Name: NVDA
Date: 06/21/2024
Type: Call
Price: 1,000.00
Estimated Yield: 0.00%
Security Name Repeat: NVDA
Account Type: Margin
Quantity: 1S
Price: 23.63
Market Value: ($2,363.00)
Market Value: $0.00
Portfolio Percentage: 13.62%
---------

In [45]:
#todo: print matches and extracted data with labels

print("Extracted Data:")
for category, values in extracted_data.items():
    print(f"Category: {category}, Values: {values}")

print("\nMatches:")
for match in matches:
    print(f"Match: {match.group(0)}")


Extracted Data:
Category: Brokerage Cash Balance, Values: ['0.00', '26.94']
Category: Deposit Sweep Balance, Values: ['260.25', '0.00']
Category: Total Securities, Values: ['0.00', '435.00']
Category: Portfolio Value, Values: ['260.25', '408.06']
Category: Cash and Cash Equivalents, Values: ['0.16']
Category: Equities, Values: ['0.00']
Category: Options, Values: ['99.84']
Category: Dividends, Values: ['0.00', '0.00']
Category: Capital Gains Distributions, Values: ['0.00', '0.00']
Category: Interest Earned, Values: ['0.00', '0.01']
Category: Stock Lending, Values: ['0.00', '0.00']

Matches:
Match: GETY 03/15/2024 Put $4.00
Estimated Yield: 0.00%GETY Margin 2 $0.10 $20.00 $0.00 0.12%
Match: NVDA 06/21/2024 Call $1,000.00
Estimated Yield: 0.00%NVDA Margin 1S $23.63 ($2,363.00) $0.00 13.62%
Match: NVDA 06/21/2024 Call $990.00
Estimated Yield: 0.00%NVDA Margin 1 $25.38 $2,538.00 $0.00 14.63%
Match: SMCI 03/15/2024 Put $860.00
Estimated Yield: 0.00%SMCI Margin 1 $58.85 $5,885.00 $0.00 33.92%

In [57]:

import os
import sqlite3

# Close any existing connections to the database
if 'conn' in globals():
    conn.close()

# Remove the existing database file if it exists
if os.path.exists('portfolio.db'):
    os.remove('portfolio.db')

# Connect to the new database (or create it if it doesn't exist)
conn = sqlite3.connect('portfolio.db')
# Create cursor object
cursor = conn.cursor()

# Create table for account summary
cursor.execute('''CREATE TABLE IF NOT EXISTS account_summary (
                    id INTEGER PRIMARY KEY,
                    category TEXT,
                    opening_balance REAL,
                    closing_balance REAL
                )''')

# Create table for income and expense summary
cursor.execute('''CREATE TABLE IF NOT EXISTS income_expense_summary (
                    id INTEGER PRIMARY KEY,
                    category TEXT,
                    this_period REAL,
                    year_to_date REAL
                )''')

# Create table for portfolio allocation
cursor.execute('''CREATE TABLE IF NOT EXISTS portfolio_allocation (
                    id INTEGER PRIMARY KEY,
                    category TEXT,
                    percentage REAL
                )''')

# Create table for portfolio details
cursor.execute('''CREATE TABLE IF NOT EXISTS portfolio_details (
                    id INTEGER PRIMARY KEY,
                    security_name TEXT,
                    date TEXT,
                    type TEXT,
                    price REAL,
                    estimated_yield REAL,
                    account_type TEXT,
                    quantity REAL,
                    price2 REAL,
                    market_value REAL,
                    portfolio_percentage REAL
                )''')

# Commit changes and close connection
conn.commit()


In [58]:
#TODO: INSERT VALUES PROPERLY

conn = sqlite3.connect('portfolio.db')
cursor = conn.cursor() 

# print("ACCOUNT SUMMARY") 
# for category, values in extracted_data.items():
#     if values is not None and len(values) == 2:
#         cursor.execute("INSERT INTO account_summary (category, opening_balance, closing_balance) VALUES (?, ?, ?)", (category, values[0], values[1]))
#     else:
#         print(f"Skipping CATEGORY {category}, {values} as it does not have the required number of values 2.")

# print("INCOME EXPENSE SUMMARY")
# for category, values in extracted_data.items():
#     if values is not None and len(values) == 2:
#         cursor.execute("INSERT INTO income_expense_summary (category, this_period, year_to_date) VALUES (?, ?, ?)", (category, values[0], values[1]))
#     else:
#         print(f"Skipping CATEGORY {category}, {values} as it does not have the required number of values 2.")


# print("PORFOLIO ALLOCATION")
# for category, value in extracted_data.items():
#     if value is not None and "Percentage" in category:
#         cursor.execute("INSERT INTO portfolio_allocation (category, percentage) VALUES (?, ?)", (category, value))
#     else:
#         print(f"Skipping CATEGORY {category}, {value} as it does not have the required number of values 1.")

# Insert data into portfolio details table
for match in matches:
    if match is not None and match.group(1):
        cursor.execute("INSERT INTO portfolio_details (security_name, date, type, price, estimated_yield, account_type, quantity, price2, market_value, portfolio_percentage) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                   (match.group(1), match.group(2), match.group(3), match.group(4), match.group(5), match.group(7), match.group(8), match.group(9), match.group(10), match.group(12)))
    else:
        print(f"Skipping STOCK MATCH {match} as it does not have the required number of values.")
    

# Commit changes and close connection
conn.commit()
conn.close()

In [60]:
conn = sqlite3.connect('portfolio.db')
cursor = conn.cursor() 

print("ACCOUNT SUMMARY")
for category in ["Brokerage Cash Balance", "Deposit Sweep Balance", "Total Securities", "Portfolio Value"]:
    if extracted_data[category] is not None and len(extracted_data[category]) == 2:
        cursor.execute("INSERT INTO account_summary (category, opening_balance, closing_balance) VALUES (?, ?, ?)", (category, extracted_data[category][0], extracted_data[category][1]))
    else:
        print(f"Skipping CATEGORY {category}, {extracted_data[category]} as it does not have the required number of values.")

print("INCOME EXPENSE SUMMARY")
for category in ["Dividends", "Capital Gains Distributions", "Interest Earned", "Stock Lending"]:
    if extracted_data[category] is not None and len(extracted_data[category]) == 2:
        cursor.execute("INSERT INTO income_expense_summary (category, this_period, year_to_date) VALUES (?, ?, ?)", (category, extracted_data[category][0], extracted_data[category][1]))
    else:
        print(f"Skipping CATEGORY {category}, {extracted_data[category]} as it does not have the required number of values.")

print("PORTFOLIO ALLOCATION")
for category in ["Cash and Cash Equivalents", "Equities", "Options"]:
    if extracted_data[category] is not None and len(extracted_data[category]) == 1:
        cursor.execute("INSERT INTO portfolio_allocation (category, percentage) VALUES (?, ?)", (category, extracted_data[category][0]))
    else:
        print(f"Skipping CATEGORY {category}, {extracted_data[category]} as it does not have the required number of values.")

conn.commit()
conn.close()

ACCOUNT SUMMARY
INCOME EXPENSE SUMMARY
PORTFOLIO ALLOCATION


In [62]:
# conn = sqlite3.connect('portfolio.db')
# cursor = conn.cursor() 
# print("ACCOUNT SUMMARY")
# account_summary_categories = ["Brokerage Cash Balance", "Deposit Sweep Balance", "Total Securities", "Portfolio Value"]
# for category in account_summary_categories:
#     values = extracted_data.get(category)
#     if values is not None and len(values) == 2:
#         cursor.execute("INSERT INTO account_summary (category, opening_balance, closing_balance) VALUES (?, ?, ?)", (category, values[0], values[1]))
#     else:
#         print(f"Skipping CATEGORY {category}, {values} as it does not have the required number of values.")

# print("INCOME EXPENSE SUMMARY")
# income_expense_categories = ["Dividends", "Capital Gains Distributions", "Interest Earned", "Stock Lending"]
# for category in income_expense_categories:
#     values = extracted_data.get(category)
#     if values is not None and len(values) == 2:
#         cursor.execute("INSERT INTO income_expense_summary (category, this_period, year_to_date) VALUES (?, ?, ?)", (category, values[0], values[1]))
#     else:
#         print(f"Skipping CATEGORY {category}, {values} as it does not have the required number of values.")

# print("PORTFOLIO ALLOCATION")
# allocation_categories = ["Cash and Cash Equivalents", "Equities", "Options"]
# for category in allocation_categories:
#     value = extracted_data.get(category)
#     if value is not None and len(value) == 1:
#         cursor.execute("INSERT INTO portfolio_allocation (category, percentage) VALUES (?, ?)", (category, value[0]))
#     else:
#         print(f"Skipping CATEGORY {category}, {value} as it does not have the required number of values.")

# conn.commit()
# conn.close()

ACCOUNT SUMMARY
INCOME EXPENSE SUMMARY
PORTFOLIO ALLOCATION


In [64]:
conn = sqlite3.connect('portfolio.db')
cursor = conn.cursor()

# Display data from account_summary table
print("Account Summary:")
cursor.execute("SELECT * FROM account_summary")
for row in cursor.fetchall():
    print(row)

# Display data from income_expense_summary table
print("\nIncome and Expense Summary:")
cursor.execute("SELECT * FROM income_expense_summary")
for row in cursor.fetchall():
    print(row)

# Display data from portfolio_allocation table
print("\nPortfolio Allocation:")
cursor.execute("SELECT * FROM portfolio_allocation")
for row in cursor.fetchall():
    print(row)

# Display data from portfolio_details table
print("\nPortfolio Details:")
cursor.execute("SELECT * FROM portfolio_details")
for row in cursor.fetchall():
    print(row)

# Close the connection
conn.close()

#todo print with proper labels

Account Summary:
(1, 'Brokerage Cash Balance', 0.0, 26.94)
(2, 'Deposit Sweep Balance', 260.25, 0.0)
(3, 'Total Securities', 0.0, 435.0)
(4, 'Portfolio Value', 260.25, 408.06)
(5, 'Brokerage Cash Balance', 0.0, 26.94)
(6, 'Deposit Sweep Balance', 260.25, 0.0)
(7, 'Total Securities', 0.0, 435.0)
(8, 'Portfolio Value', 260.25, 408.06)

Income and Expense Summary:
(1, 'Dividends', 0.0, 0.0)
(2, 'Capital Gains Distributions', 0.0, 0.0)
(3, 'Interest Earned', 0.0, 0.01)
(4, 'Stock Lending', 0.0, 0.0)
(5, 'Dividends', 0.0, 0.0)
(6, 'Capital Gains Distributions', 0.0, 0.0)
(7, 'Interest Earned', 0.0, 0.01)
(8, 'Stock Lending', 0.0, 0.0)

Portfolio Allocation:
(1, 'Cash and Cash Equivalents', 0.16)
(2, 'Equities', 0.0)
(3, 'Options', 99.84)
(4, 'Cash and Cash Equivalents', 0.16)
(5, 'Equities', 0.0)
(6, 'Options', 99.84)

Portfolio Details:
(1, 'GETY', '03/15/2024', 'Put', 4.0, 0.0, 'Margin', 2.0, 0.1, '$20.00', 0.12)
(2, 'NVDA', '06/21/2024', 'Call', '1,000.00', 0.0, 'Margin', '1S', 23.63, '(

In [ ]:
#data validation?

In [ ]:
#display in webapp

In [ ]:
# Not only project the current summary, winners, and losers, but run stimulations on timelines, strike price, and (correlated) ticker choices for options and stocks.
# Will require external API for stock data and perhaps some homegrown math